# Circuit Training - QAOA
## [QHack 2021](https://challenge.qhack.ai/)
### Matt Wright

In [1]:
import networkx as nx
import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt

In [2]:
NODES = 6
N_LAYERS = 10

In [3]:
def find_max_independent_set(graph, params):
    """Find the maximum independent set of an input graph given some optimized QAOA parameters.

    Args:
        graph (nx.Graph): A NetworkX graph
        params (np.ndarray): Optimized QAOA parameters of shape (2, 10)

    Returns:
        list[int]: the maximum independent set, specified as a list of nodes in ascending order
    """

    # My solution begins here #
    wires = range(NODES)
    dev = qml.device('default.qubit', wires=wires)
    
    cost_h, mixer_h = qml.qaoa.max_independent_set(graph)
    def qaoa_layer(gamma, alpha):
        qml.qaoa.cost_layer(gamma, cost_h)
        qml.qaoa.mixer_layer(alpha, mixer_h)
    
    @qml.qnode(dev)
    def probability_circuit(params):
        qml.layer(qaoa_layer, N_LAYERS, params[0], params[1])
        return qml.probs(wires=wires)
    
    probs = probability_circuit(params)
    
    max_prob = max(list(probs))
    max_index = list(probs).index(max_prob)
 
    binary_idx = "{0:b}".format(max_index)
    binary_idx = '0' * (NODES-len(binary_idx)) + binary_idx
    
    max_ind_set = np.array([int(i) for i, v in enumerate(binary_idx) if v != '0'])
    
    # My solution ends here #
    return max_ind_set

## Testing

In [4]:
from test_result import test_result

In [5]:
tol = 10e-3

print('Test 1:')
graph_data = {"directed": False, "multigraph": False, "graph": [], "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}], "adjacency": [[{"id": 2}, {"id": 3}], [{"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}], [{"id": 0}, {"id": 1}], [{"id": 0}, {"id": 1}, {"id": 5}], [{"id": 1}, {"id": 5}], [{"id": 1}, {"id": 3}, {"id": 4}]], "params": [[0.007251195968545696, 0.0008594067983382697, 6.757788685303326e-05, -0.002960663108058259, -3.990107834181506e-05, 0.0008459551312108134, -0.001277886948034289, 0.0013917849664037338, 0.0050844175570855495, 0.0004046899603893801], [0.056410689341680226, 0.07332551003086096, 0.11573770360937545, 0.1471947383136579, 0.16520379259436668, 0.16107577283133256, 0.13346398582028385, 0.08533445938002224, 0.06505979609496622, 0.047259251575614024]]}
params = np.array(graph_data.pop("params"))
graph = nx.json_graph.adjacency_graph(graph_data)

pred_1 = find_max_independent_set(graph, params)
ans_1 = np.array([2, 3, 4])
test_result(pred_1, ans_1, tol)

print('\nTest 2:')
graph_data = {"directed": False, "multigraph": False, "graph": [], "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}], "adjacency": [[{"id": 3}, {"id": 4}], [{"id": 4}, {"id": 5}], [{"id": 3}, {"id": 5}], [{"id": 0}, {"id": 2}, {"id": 4}], [{"id": 0}, {"id": 1}, {"id": 3}], [{"id": 1}, {"id": 2}]], "params": [[0.00725119586350819, 0.0006993146552754333, 0.00018938653033438184, -0.002576694936729869, 0.000184128566739523, 0.0007971715773060028, -0.001474030227059991, 0.001133453129607066, 0.004748509435854284, -2.719819887597862e-05], [0.023501757757741343, 0.04041058176918198, 0.10033413722212867, 0.1891521087372252, 0.24644283314811008, 0.2038888864247133, 0.12928227840808193, 0.06257051158195509, 0.04319572071910715, 0.02744074527290291]]}
params = np.array(graph_data.pop("params"))
graph = nx.json_graph.adjacency_graph(graph_data)

pred_2 = find_max_independent_set(graph, params)
ans_2 = np.array([0, 1, 2])
test_result(pred_2, ans_2, tol)

Test 1:
Success!!

Test 2:
Success!!
